In [119]:
import pandas as pd
import sqlite3 as sql
import nltk

In [144]:
conn = sql.connect("track_metadata.db")
# conn2 = sql.connect("dat/mxm_dataset.db")
c = conn.cursor()

In [145]:
for row in c.execute("SELECT name FROM main.sqlite_master WHERE type='table';"):
    print(row)
    

('songs',)


In [146]:
c.execute("attach 'mxm_dataset.db' AS lyr;")
c.execute("attach 'lastfm_tags.db' AS tag;") 

In [147]:
for row in c.execute("SELECT name FROM lyr.sqlite_master WHERE type='table';"):
    print(row)

('words',)
('lyrics',)


In [148]:
for row in c.execute("SELECT name FROM tag.sqlite_master WHERE type='table';"):
    print(row)

('tags',)
('tids',)
('tid_tag',)


In [149]:
q = '''
SELECT * FROM songs
LIMIT 200;
'''

In [150]:
q2 = '''
SELECT * FROM lyrics
LIMIT 200;
'''

In [151]:
c1 = '''
ATTACH DATABASE "dat/mxm_dataset.db" AS conn3;
'''

In [152]:
c2 = '''
ATTACH DATABASE "dat/artist_term.db" AS conn3;
'''

In [153]:
songs = pd.read_sql("SELECT * FROM main.songs", conn)
lyrics = pd.read_sql("SELECT * FROM lyr.lyrics LIMIT 100", conn)
genres = pd.read_sql("SELECT * FROM tag.tags LIMIT 100", conn)
gtags = pd.read_sql("SELECT * FROM tag.tid_tag LIMIT 100", conn)

# Cleaning Begins!

In [154]:
#artist name, song title, release, all lowercase
songs["title"] = songs["title"].str.lower()
songs["release"] = songs["release"].str.lower()
songs["artist_name"] = songs["artist_name"].str.lower()

#drops artist hotttness metric; drops na vals
songs = songs.drop(columns = ["artist_hotttnesss", "track_7digitalid", "shs_perf", "shs_work", "artist_mbid"])
songs = songs.dropna()

In [155]:

#trim songs with years before 1975
songs = songs[songs.year > 1975]

#renaming the title column to "song_name" and rename "release" to "album_name"
songs = songs.rename(columns={"title":"song_name", "release":"album_name"})

In [156]:
#dropping based on familiarity being below 20
songs = songs[songs.artist_familiarity > .2]
songs = songs.sort_values(by = "artist_familiarity", ascending = False)

#removes rows with blank song entries
songs = songs[songs.song_name.str.len() > 1].sort_values(by="song_name")

#removes rows with years before 1975
songs = songs[songs.year > 1975]

print(len(songs))

482823


In [157]:
#duplicate removal--removes all duplicate songs by matching artist id

################## Note: this step removes duplicates when song_name and artist_id are exactly the same
################## but not different versions of a same song

songs = songs.sort_values(by = "artist_name")
songs["s/a"] = songs["song_name"]+songs["artist_id"] #song-artist concatenation column
songs = songs.drop_duplicates("s/a", keep = 'first')
print(len(songs))

438616


In [158]:
#######check############
print(list(songs.columns.values))
print(min(songs['year']),max(songs['year']))

['track_id', 'song_name', 'song_id', 'album_name', 'artist_id', 'artist_name', 'duration', 'artist_familiarity', 'year', 's/a']
1976 2011


 # TODO: if alternate versions of songs exist, keep the original only

In [159]:
import re
pattern = r' \(.*?version\)'

In [160]:
dump = re.search(pattern, '(bad version) (better version)')

In [161]:
dump.span(0)

(13, 30)

In [162]:
dump = songs['song_name'].map(lambda s: len(list(re.finditer(pattern, s))))

In [163]:
import numpy as np
dump = np.asarray(dump)
print((dump <= 1).sum())
print(len(dump))

438582
438616


In [164]:
#IMPLIMENT IRENE!
#test: see how different versions are represented
#first, try to find some examples
#songs.groupby('artist_id').count().sort_values(by='song_name', ascending=False)
#checkout the artist with the most entry
#songs.loc[songs['song_name'].str.contains('version')].sort_values(by='song_name', ascending=False)
#drop all the (xx version) part of the song name and see if the remaining string still has duplicates
def clean_string(s):
    match_obj = re.search(pattern, s)
    if match_obj is None:
        return s
    start, end = match_obj.span(0)
    return s[:start] + s[end:]

songs['song_name_cleaned'] = songs['song_name'].map(clean_string)

In [165]:
print(len(songs))
songs['s_cleaned/a'] = songs["song_name_cleaned"]+songs["artist_id"]
songs_test = songs.sort_values(by='s_cleaned/a')
songs_test = songs.drop_duplicates("s_cleaned/a", keep = 'first')
print(len(songs_test))

438616
436690


In [167]:
#so there are duplicates! Even after dropping the "version" part! But not that many!
#for now I am going to just keep the OLDEST of uplicates, regardless of which version it is
#do we need to keep specific version? say, favor EP version to album version?
songs = songs.sort_values(['s_cleaned/a', 'year'])
songs = songs.drop_duplicates("s_cleaned/a", keep = 'first')
print(len(songs))

436690


In [170]:
songs.head(10)

,track_id,song_name,song_id,album_name,artist_id,artist_name,duration,artist_familiarity,year,s/a,song_name_cleaned,s_cleaned/a
548988,TRTBNCR128F424378D,till kingdom come,SORLDBM12A8C1335D6,old growth,ARNEMNT1187B9B44B1,dead meadow,245.99465,0.690974,2008,till kingdom comeARNEMNT1187B9B44B1,till kingdom come,till kingdom comeARNEMNT1187B9B44B1
129025,TRHZQPU128F4248D91,ef ég hefði aldrei...,SOKBHBV12A8C130430,englabörn,ARXO7AU1187B9B249F,johann johannsson,222.87628,0.625280,2002,ef ég hefði aldrei...ARXO7AU1187B9B249F,ef ég hefði aldrei...,ef ég hefði aldrei...ARXO7AU1187B9B249F
290346,TRFTRGQ128F4248D70,eg sleppi þer aldrei,SOVHCGB12A8C1303EA,englabörn,ARXO7AU1187B9B249F,johann johannsson,177.94567,0.625280,2002,eg sleppi þer aldreiARXO7AU1187B9B249F,eg sleppi þer aldrei,eg sleppi þer aldreiARXO7AU1187B9B249F
584850,TRUCXGV128F422BB0D,"the arsonist story"": evil craves attention/o...",SOQOVFQ12A6D4FAE4B,1212,AR9HLNV1187B98EFC6,barbara manning,485.61587,0.491941,1997,"the arsonist story"": evil craves attention/o...","the arsonist story"": evil craves attention/o...","the arsonist story"": evil craves attention/o..."
343266,TRQXVIR128F422BB0B,"the arsonist story"": evil plays piano",SOTRINY12A6D4FAE46,1212,AR9HLNV1187B98EFC6,barbara manning,41.24689,0.491941,1997,"the arsonist story"": evil plays pianoAR9HLNV...","the arsonist story"": evil plays piano","the arsonist story"": evil plays pianoAR9HLNV..."
530024,TRPSEYQ128F422BB12,"the arsonist story"": trapped and drowning",SOUZGEF12A6D4FAE62,1212,AR9HLNV1187B98EFC6,barbara manning,420.75383,0.491941,1997,"the arsonist story"": trapped and drowningAR9...","the arsonist story"": trapped and drowning","the arsonist story"": trapped and drowningAR9..."
972174,TRYFMAC128F422BB08,"the arsonist story"": fireman",SORQBMP12A6D4FAE43,1212,AR9HLNV1187B98EFC6,barbara manning,177.05751,0.491941,1997,"the arsonist story"": firemanAR9HLNV1187B98EFC6","the arsonist story"": fireman","the arsonist story"": firemanAR9HLNV1187B98EFC6"
138487,TRHUSPZ128EF35F572,why im a lawbreaker,SOYBKQK12A6D4F66CA,weaponry listens to love,AR20Z421187FB40C07,huggy bear,84.66240,0.552386,1994,why im a lawbreakerAR20Z421187FB40C07,why im a lawbreaker,why im a lawbreakerAR20Z421187FB40C07
963984,TRYWPZY128F4248D80,ég heyrði allt án þess að hlusta,SORXLSH12A8C13040A,englabörn,ARXO7AU1187B9B249F,johann johannsson,125.04771,0.625280,2002,ég heyrði allt án þess að hlustaARXO7AU1187B9...,ég heyrði allt án þess að hlusta,ég heyrði allt án þess að hlustaARXO7AU1187B9...
517592,TRPAQJI128F4248D8D,ég átti gráa æsku,SOUDIUF12A8C130429,englabörn,ARXO7AU1187B9B249F,johann johannsson,221.28281,0.625280,2002,ég átti gráa æskuARXO7AU1187B9B249F,ég átti gráa æsku,ég átti gráa æskuARXO7AU1187B9B249F


# Begin Merging Genre and Lyrics Section

In [174]:
#this section is to determine the most popular genre labels

"""sg = pd.read_sql("SELECT tag.tids.tid, tag.tags.tag FROM tag.tid_tag \
            INNER JOIN tag.tags ON tag.tags.ROWID==tag.tid_tag.tag \
            INNER JOIN tag.tids ON tag.tid_tag.tid==tag.tids.ROWID \
             LIMIT 1000000", conn)"""

sg = pd.read_sql("SELECT tag.tids.tid, tag.tags.tag FROM tag.tid_tag \
            INNER JOIN tag.tags ON tag.tags.ROWID==tag.tid_tag.tag \
            INNER JOIN tag.tids ON tag.tid_tag.tid==tag.tids.ROWID \
             ", conn)

data = pd.DataFrame()
sg['tag'] = sg.tag.str.lower()
gdict = sg['tag'].value_counts().to_dict()
populargenres = pd.DataFrame.from_dict(gdict, orient='index', columns = ['genre_count'])

In [173]:
populargenres.columns=['genre','genre_count']

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements

In [175]:
holdgenre = populargenres.index.get_values()
populargenres["genre"] = holdgenre

In [176]:
sg_frequency=sg.merge(populargenres,left_on='tag',right_on='genre',how='left')

In [177]:
sg_frequency.head()

,tid,tag,genre_count,genre
0,TRCCCYE12903CFF0E9,classic rock,25771,classic rock
1,TRCCCYE12903CFF0E9,progressive rock,13406,progressive rock
2,TRCCCFH12903CEBC70,blues,20474,blues
3,TRCCCFH12903CEBC70,memphis slim,19,memphis slim
4,TRCCCJT128F429FFF6,pop,69159,pop


In [178]:
df_songs_merge=songs.merge(sg_frequency,left_on='track_id',right_on='tid',how='left')

In [179]:
df_songs_merge=df_songs_merge[~df_songs_merge['tag'].isna()]

In [38]:
df_songs_merge

,track_id,song_name,song_id,album_name,artist_id,artist_name,duration,artist_familiarity,year,s/a,tid,tag,genre_count,genre
13,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,great covers,443.0,great covers
14,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,for my dancy mix,3.0,for my dancy mix
15,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,stereo cool,65.0,stereo cool
16,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,dream for me please,5.0,dream for me please
17,TRPMYJW128F9323194,camera,SOCKIWT12AB017F1A6,a house full of friends,ARI31A31187B9A7B6C,#poundsign#,391.00036,0.391269,2005,cameraARI31A31187B9A7B6C,TRPMYJW128F9323194,thissongmakesmewanttocrynotafraidtosharemyemot...,1.0,thissongmakesmewanttocrynotafraidtosharemyemot...
18,TRHAEMQ128F427C025,ayso,SOOWKJU12A58A7A6F2,sevensummers - international pop volume two,ARI31A31187B9A7B6C,#poundsign#,246.12526,0.391269,2000,aysoARI31A31187B9A7B6C,TRHAEMQ128F427C025,happy,12291.0,happy
19,TRHAEMQ128F427C025,ayso,SOOWKJU12A58A7A6F2,sevensummers - international pop volume two,ARI31A31187B9A7B6C,#poundsign#,246.12526,0.391269,2000,aysoARI31A31187B9A7B6C,TRHAEMQ128F427C025,-,53.0,-
20,TRHAEMQ128F427C025,ayso,SOOWKJU12A58A7A6F2,sevensummers - international pop volume two,ARI31A31187B9A7B6C,#poundsign#,246.12526,0.391269,2000,aysoARI31A31187B9A7B6C,TRHAEMQ128F427C025,the office girls,98.0,the office girls
23,TRJCBRC128F92D0FBC,todas las noches,SOKQBUR12A8C143F52,day by day,ARQ3NFE1187FB4801D,$tinkworx,253.93587,0.327675,2002,todas las nochesARQ3NFE1187FB4801D,TRJCBRC128F92D0FBC,future funk,39.0,future funk
24,TRWJHXP128F92D0FA2,creeping thru lower levels,SOHULMD12A8C140603,ain't-chit history,ARQ3NFE1187FB4801D,$tinkworx,237.71383,0.327675,2001,creeping thru lower levelsARQ3NFE1187FB4801D,TRWJHXP128F92D0FA2,808,83.0,808


# We just have to find a way to impute these vals now!

alternative rock -->aleternative
hip-hop ---> hiphop
hip hop----> hiphop
hiphop
folk
pop
rock
indie
indie rock ---> indie
metal
heavy metal ---> metal
jazz
classic rock
elctronic 
electrocica ---> elctronic
house ---> electronic
techno ---> electronic
trance ---> electronic
electro --->electronic
soul
rap
country
blues
punk 
o/w ---> drop

In [180]:
#first, sanity check
print(len(df_songs_merge))

5134665


In [191]:
def merge_genre(s):
    name_mapping = {
        'alternative rock': 'rock',
        'hip-hop': 'hiphop',
        'hip hop': 'hiphop',
        'indie rock': 'indie',
        'heavy metal': 'metal',
        'electrocica': 'elctronic',
        'house':'electronic',
        'techno': 'electronic',
        'trance': 'electronic',
        'electro': 'electronic'}
    
    keep_set = {
        'hiphop',
        'folk',
        'pop',
        'rock',
        'indie',
        'metal',
        'metal',
        'jazz',
        'classic rock',
        'electronic',
        'soul',
        'rap',
        'country',
        'blues',
        'punk'
    }
    if s in keep_set:
        return s
    elif s in name_mapping:
        return name_mapping[s]
    else:
        return None

df_songs_merge['genre_merged'] = df_songs_merge['genre'].map(merge_genre)
print(len(df_songs_merge))
tmp = df_songs_merge.dropna()
print(len(tmp))

5134665
381087


In [194]:
#number of songs in each genre
tmp1 = tmp.groupby('genre_merged').count()
print(tmp1)

              track_id  song_name  song_id  album_name  artist_id  \
genre_merged                                                        
blues             8946       8946     8946        8946       8946   
classic rock     10839      10839    10839       10839      10839   
country           8021       8021     8021        8021       8021   
electronic       55227      55227    55227       55227      55227   
folk             15100      15100    15100       15100      15100   
hiphop           20936      20936    20936       20936      20936   
indie            54052      54052    54052       54052      54052   
jazz             16558      16558    16558       16558      16558   
metal            29365      29365    29365       29365      29365   
pop              40943      40943    40943       40943      40943   
punk             14747      14747    14747       14747      14747   
rap               7086       7086     7086        7086       7086   
rock             87236      87236 